# ANOVA mit Messwiederholungen

In dem letzten Teil haben wir uns ANOVAs angesehen, die sich mit einem Faktor ("Culmen Length (mm)") zwischen verschiedenen Individuuen beschäftigt. Also einem sogenannten between subjects factor. Die Variation zwischen den Gruppen also der between factor war der für uns interessante Effekt. Was ist aber, wenn wir wiederholt dieselbe Person messen, dann können wir nicht ohne weiteres dieselbe Logik anwenden, da wir die Variiation innerhalb der Gruppen, den within subjects factor, einfach als Fehler angesehen haben? Das wir das beim letzten Mal einfach so gemacht haben, verdeutlicht die Formel für den F-Wert.


$$ F = \frac{Signal}{Noise} = \frac{Explained\:variance}{Unexplained\:variance} = \frac{Between\:group\:variability}{Within\:group\:variability} $$

Wenn wir allerdings Messwiederholungen haben, sind in jeder Gruppe dieselben Individuuen und allein dadurch lässt sich einiges der Variablität innerhalb der Gruppen erklären. Wir teilen also die Within group Varianz im Nenner weiter auf. 

$$Within\:group\:variability = Subject\:variability + Error$$

Da die durch die einzelnen Subjects erklärte Variabliltät (und es sich ja um erklärte Varianz handelt) nichts zur Frage beiträgt, ob über die Zeit/verschiedene Interventionen zu unterschiedlichen Werten führt, wird die Formel für den F-Wert angepasst für die ANOVA mit Messwiederholungen.

$$ F = \frac{Signal}{Noise} = \frac{Explained\:variance}{Unexplained\:variance} = \frac{Between\:group\:variability}{Error} $$

Unser Pinugindatensatz enthält allerdings keine Messwiederholungen, wie wir sehen, wenn wir ihn uns nochmal ansehen

In [ ]:
import pandas as pd

penguins = pd.read_csv("./penguins_classification.csv")

penguins.head()

In der nächsten Zelle passiert ziemlich viel Code, der nur dazu da ist unseren zweiten und dritten Zeitpunkt zu generieren und unseren Dataframe für eine repeated ANOVA vorzubereiten. Hierfür müssen wir verschiedene Dinge beachten. Eine repeated ANOVA braucht zum Beispiel einen Identifier, um zu wissen, welcher Wert zu welchem Individuum gehört (in unseren Simulierten Daten ändert sich die "Culmen Depth (mm)" nicht über die Zeit und die Species der Pinguine bleibt auch konstant, daher ist der Code für diese beiden parameter sehr ähnlich wie für die Identifier). Außerdem brauchen wir eine Variable (z.B. "time"), die den Messzeitpunkt kodiert und außerdem die Variable "Culmen Length (mm)", die sich über unsere drei Messzeitpunkte verändern soll. Dafür füge ich zu dem ursprünglichen Wert aus unseren Daten einen zufällig generierten Wert hinzu, um Schnabelwachstum zu simulieren. 

Ihr müsst nicht 100% verstehen, was in dieser Zelle passiert. Am allerwichtigsten sind die letzten zwei Zeile. Auch hier passiert ziemlich viel, wenn man sie Stück für Stück durchgeht wird es aber alles klar.

1. Wir entfernen die Spalten "Species" und "id" aus unserem neu erstellten DataFrame penguins_rm. 
2. Wir Gruppieren die Werte von "Culmen Length (mm)" und "Culmen Depth (mm)" nach der Zeit und lassen uns für beides den Mittelwert und die Standardabweichung ausgeben. Das ist quasi eine Weitere Methode um solche Übersichten wie mit `.describe` zu erstellen. Das könnte man auch statt `.agg(["mean", "std"])` anhängen. Die vielen Outputs von `.describe` sind mir hier allerdings zu unübersichtlich 

In [ ]:
## generate t2 and t3

import numpy as np
# np.random.rand generiert einen zufälligen Vektor mit den in den Klammern angegebenen Dimensionen
penguins_rm = pd.DataFrame({
    "id": np.resize(np.arange(0, len(penguins)), 3*len(penguins)), # Id erstellen und dreimal wiederholen, da wir drei Messzeitpunkte haben
    "time": ["t1"] * len(penguins) + ["t2"] * len(penguins) + ["t3"] * len(penguins), # 
    "Culmen Length (mm)": pd.concat([
        penguins["Culmen Length (mm)"],
        penguins["Culmen Length (mm)"] + np.random.rand(len(penguins)),
        penguins["Culmen Length (mm)"] + 0.5 + np.random.rand(len(penguins))
        ]),
    "Culmen Depth (mm)": np.resize(penguins["Culmen Depth (mm)"], 3*len(penguins)),
    "Species": np.resize(penguins["Species"], 3*len(penguins))
})

penguins_rm_numerical = penguins_rm.drop(columns=["Species", "id"])
penguins_rm_numerical.groupby(["time"]).agg(["mean", "std"])

Mit dem Datensatz können wir nun eine ANOVA mit Messwiederholungen rechnen. Die Documentation gibt uns wieder einen Hinweis, was die einzelnen Arguments bedeuten.

In [ ]:
import pingouin as pg

?pg.rm_anova

In [ ]:
rm_anova_results = pg.rm_anova(
    data=penguins_rm,
    dv="Culmen Length (mm)",
    within="time",
    subject="id",
    detailed=True,
)

rm_anova_results = rm_anova_results.set_index("Source")
rm_anova_results.round(3) # .round(3) rundet die ANOVA-Ergebnisse auf drei Nachkommastellen

## Sums of Square

Hier können wir am einfachsten auch wieder die $SS_{gesamt}$ ausrechnen. Also die gesamte in unserer Stichprobe vorhandene Variablität. Allerdings lässt sich das errechnete Ergebnis nicht mehr so einfach auf seine Richtigkeit prüfen, indem wir alles SS aus der Tabelle des ANOVA Outputs zusammenrechnen. Das liegt vor allem daran, dass die $SS_{within}$ bei einer ANOVA mit Messwiederholungen weiter aufgeteilt werden, da in jeder Gruppe dieselben Individuen sind. 

Q: Kannst du die $SS_{gesamt}$ errechnen? 

In [ ]:
# Lösung
x = penguins_rm["Culmen Length (mm)"]
M_gesamt = np.mean(x)

SS_gesamt = np.sum( (x - M_gesamt) ** 2)
SS_gesamt

### $SS_{within}$

Die Berechnung der SS within ist analog zur ANOVA ohne Messwiederholungen. Wir nehmen unseren drei Gruppen berechnen für jede innerhalb jeder Gruppe die Summe des quadrierten Mittelwerts und summieren diese wieder auf.

In [ ]:
def SumOfSquares(input_list):
    """
    input_list - Liste mit Daten für die, die Sum of Squares berechnet werden soll.
    """
    x = input_list.to_numpy() # .to_numpy macht aus dem Pandas Dataframe/Series etwas womit numpy gut rechnen kann
    Mx = np.mean(x)
    
    return np.sum((x - Mx) ** 2)

In [ ]:
SS_t1 = SumOfSquares(penguins_rm[penguins_rm["time"] == "t1"]["Culmen Length (mm)"])
SS_t2 = SumOfSquares(penguins_rm[penguins_rm["time"] == "t2"]["Culmen Length (mm)"])
SS_t3 = SumOfSquares(penguins_rm[penguins_rm["time"] == "t3"]["Culmen Length (mm)"])

SS_within = SS_t1 + SS_t2 + SS_t3
SS_within

Dieser Wert kommt uns erstmal gar nicht bekannt vor aus der Tabelle oben. Aber wenn wir kurz an die Formel von oben denken ist das gar nicht so verwunderlich. Schließlich ist für die ANOVA mit Messwiederholungen nicht mehr die $Variablität_{within}$ sondern des Fehlers entscheidend:


$$ F = \frac{Signal}{Noise} = \frac{Explained\:variance}{Unexplained\:variance} = \frac{Between\:group\:variability}{Error} $$

Die ersten uns bekannt vorkommenden Werte sollten wir also beim Ausrechnen der $SS_{between}$ erkennen.

### $SS_{between}$

Die $SS_{between}$ errechnen wir wieder entweder indem wir die $SS_{within}$ von der $SS_{gesamt}$ abziehen oder indem wir eine SS mit den Gruppenmittelwerten, die um den Gesamtmittelwert variieren ausrechnen. Zuerst prüfen wir die erste Version, um zu gucken, ob die Werte in der Tabelle oben vorkommen:

In [ ]:
SS_between1 = SS_gesamt - SS_within
print(
    f"SS_between aus der rm ANOVA-Tabelle: {rm_anova_results['SS']['time']:.3f}\n"
    f"SS_between errechnet: {SS_between1:.3f}"
)

Vielleicht werdet ihr an diesem Punkt etwas stutzig. War time in dem Code zur Berechnung der ANOVA nicht der within factor?
```
rm_anova_results = pg.rm_anova(
    data=penguins_rm,
    dv="Culmen Length (mm)",
    within="time",
    subject="id",
    detailed=True,
)
```
Das ist vollkommen richtig! Bei dem within factor geht es um den Faktor den alle Individuuen ausgesetzt werden, z.B. Messungen über die Zeit, bei der die Individuen gleich bleiben oder Cross-Over Designs wo die dieselben Individuuen verschiedene Interventionen bekommen. Bei der $SS_{between}$ geht es um die Variation zwischen den Gruppen. Wir befinden uns also auf einer unterschiedlichen Ebene. Solche Wiederverwendung von Begrifflichkeiten kommen leider häufiger in der Statistik vor und können oft zu Verwirrung führen.


Q: Kannst du die zweite Variante selbstständig errechnen? Falls du Hilfe brauchst kannst du nochmal in **03_ANOVA_Hintergründe** gucken.

In [ ]:
# Lösung
group_means = penguins_rm.groupby("time").mean()["Culmen Length (mm)"]
group_n = penguins_rm.groupby("time").count()["Culmen Length (mm)"]
SS_between = np.sum( group_n * (group_means - M_gesamt) ** 2)
SS_between

### $SS_{subj*}$

Jetzt müssen wir nur noch herausfinden, welchen Anteil der $SS_{within}$ durch die $SS_{subj*}$ erklärt wird, da wir die Within Varianz weiter aufteilen in Fehler und Variation, die dadurch erklärt wird, dass wir Individuen mehrfach messen.

Hierfür nutzen wir eine Ähnliche Logik wie bei den $SS_{between}$. Erinnert euch an die Formel für die $SS_{between}$:

$$SS_{between} = \sum n_i(M_i - M_{gesamt})^2$$

Jetzt ersetzen wir den Gruppenmittelwert $M_i$ durch den Mittelwert der einzelnen Individuen über die verschiedenen Messzeitpunkte und $n_i$, die Anzahl der Personen in den Gruppen, durch die Messzeitpunkte $k$:

$$SS_{subj*} = \sum k(M_{subj*} - M_{gesamt})^2$$

Q: Kannst du den Code für die $SS_{between}$ so anpassen, dass die $SS_{subj*}$ berechnet wird?

In [ ]:
# Lösung
subj_means = penguins_rm.groupby("id").mean()["Culmen Length (mm)"]
k = penguins_rm["time"].nunique()
SS_subj = np.sum( k * (subj_means - M_gesamt) ** 2)
SS_subj

### $SS_{Error}$

Da $SS_{within} = SS_{subj*} + SS_{Error}$ können wir jetzt $SS_{Error}$ ausrechnen.

In [ ]:
SS_error = SS_within - SS_subj

print(
    f"SS_error aus der rm ANOVA-Tabelle: {rm_anova_results['SS']['Error']:.3f}\n"
    f"SS_error errechnet: {SS_error:.3f}"
)

## Freiheitsgrade und MSs

Wie bei der ANOVA ohne Messwiederholungen bleibt jetzt übrig die Freiheitsgrade zu errechnen, um damit die MSs zu errechnen. Die Logik ist auch weiterhin eine ähnliche. 

1. Freiheitsgrade für die gesamte ANOVA wird ausgerechnet, indem wir die Anzahl an Messwerten (N) -1 nehmen.
2. Freiheitsgrade für $SS_{between}$ indem wir die Anzahl Gruppen -1 nehmen.
3. Freiheitsgrade für $SS_{within}$ indem wir die Summe der Gruppengröße je -1 nehmen für jede Gruppe, was der $Anzahl\:an\:Messwerten\:(N) - Anzahl\:an\:Gruppen\:(k)$

In [ ]:
N = len(penguins_rm)
df_gesamt = N - 1

k = penguins_rm["time"].nunique()
df_between = k - 1
df_within = N - k 

print(
    f"df_between aus der rm ANOVA-Tabelle: {rm_anova_results['DF']['time']}\n"
    f"df_between errechnet: {df_between}")

Neu ist die Berechnung der $df_{subj}$, die Logik bleibt aber eine ähnliche. Anzahl - 1. Hier die Anzah an gemessenen Individuen. Also:

$$df_{subj} = N_{subj} - 1  

In [ ]:

N_subj = penguins_rm["id"].nunique()
df_subj = N_subj - 1

Hieraus können wir dann wie bei den SS die $df_{error}$ errechnen, denn auch hier gilt $df_{within} = df_{subj} + df_{error}$

In [ ]:
df_error = df_within - df_subj

print(
    f"df_error aus der rm ANOVA-Tabelle: {rm_anova_results['DF']['Error']}\n"
    f"df_error errechnet: {df_error}")

Wie bei den SS finden wir in der Tabelle nur die $df_{between}$ und die $df_{error}$ wieder, da diese die entscheidenden sind, um den F-Wert zu berechnen. Die MS lassen sich dann errechnen ($MS = \frac{SS}{df}$)

In [ ]:
MS_gesamt = SS_gesamt / df_gesamt
MS_between = SS_between / df_between
MS_within = SS_within / df_within
MS_subj = SS_subj / df_subj
MS_error = SS_error / df_error

print(
    f"MS_between aus der rm ANOVA-Tabelle: {rm_anova_results['MS']['time']:.3f}\n"
    f"MS_between errechnet: {MS_between:.3f}\n"
    f"MS_error aus der rm ANOVA-Tabelle: {rm_anova_results['MS']['Error']:.3f}\n"
    f"MS_error errechnet: {MS_error:.3f}")

## F-Wert

Der F-Wert wird anschließend aus der MS des interessiereden Effects und dem MS des Fehlers errechnet. Die allgemeine Formel ist also:

$$F = \frac{MS_{effect}}{MS_{error}}$$

Wir wollen sehen, ob das Schnabelwachstum über die Zeit größer ist als die unerklärte Varianz. Das heißt wir setzten $MS_{between}$ und $MS_{error}$ ein.



In [ ]:
F = MS_between / MS_error

print(
    f"F-Wert aus der rm ANOVA Tabelle: {rm_anova_results['F']['time']:.3f}\n"
    f"F-Wert errechnet: {F:.3f}"
)

Auch hier kann man wieder mit den F-Werten und den Freiheitsgraden den p-Wert aus der F-Verteilung ablesen.

Mithilfe eines Scipy Moduls können wir uns das allerdings ausgeben lassen.

In [ ]:
from scipy.stats import f

p_value = f.sf(F, df_between, df_error)
print(
    f"Errechneter P-Wert: {p_value}\n"
    f"P-Wert aus der Tabelle: {rm_anova_results['p-unc']['time']}"
)

## (Partielles) $\eta^2$

Zuletzt noch die Effektstärke. Die ändert sich im Vergleich zu vorher ein wenig, da das partielle Eta Quadrat nun einen anderen Wert produziert als das Eta Quadrat.
$$\eta^2 = \frac{SS_{effect}}{SS_{gesamt}}$$

$$partielles\:\eta^2 = \frac{SS_{effect}}{SS_{effect} + SS_{error}}$$

In [102]:
np2 = SS_between / (SS_between + SS_error)

print(f"Partilles Eta Quadrat aus der ANOVA-Tabelle: {rm_anova_results['np2']['time']:.3f}\n"
      f"Partielles Eta Quadrat errechnet: {np2:.3f}")

Partilles Eta Quadrat aus der ANOVA-Tabelle: 0.814
Partielles Eta Quadrat errechnet: 0.814
